#Exercício 2:

- Modelo de Regressão e Classificação com o Dataset Titanic

In [1]:

import h2o
from h2o.estimators import H2OGeneralizedLinearEstimator, H2ORandomForestEstimator

# Inicializar H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 36 mins
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_avlal_rc0d8l
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.910 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [2]:
# Carregar e preparar o dataset Titanic

data_titanic = h2o.import_file("..\\data\\titanic.csv" )
print("Nomes das colunas:", data_titanic.columns)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Nomes das colunas: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [3]:
# Definir a variável alvo e preditoras
target = "Survived"
predictors = [col for col in data_titanic.columns if col != target]

# Converter a variável alvo para numérica para o modelo de regressão
data_titanic[target] = data_titanic[target].asnumeric()

# Dividir em treino e teste
train, test = data_titanic.split_frame([0.75], seed=42)

In [4]:
# Modelo de Regressão

model_regression = H2OGeneralizedLinearEstimator(family="gaussian")
model_regression.train(x=predictors, y=target, training_frame=train)
performance_regression = model_regression.model_performance(test)
print("Resultados do Modelo de Regressão:")
print(performance_regression)

c:\Users\avlal\AppData\Local\Programs\Python\Python313\Lib\site-packages\h2o\estimators\estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [Name]
  warnings.warn(mesg["message"], RuntimeWarning)
c:\Users\avlal\AppData\Local\Programs\Python\Python313\Lib\site-packages\h2o\estimators\estimator_base.py:192: RuntimeWarning: We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Resultados do Modelo de Regressão:
ModelMetricsRegressionGLM: glm
** Reported on test data. **

MSE: 0.1443717009796068
RMSE: 0.379962762622348
MAE: 0.323919350189945
RMSLE: 0.2686458882704097
Mean Residual Deviance: 0.1443717009796068
R^2: 0.402228713363291
Null degrees of freedom: 227
Residual degrees of freedom: 219
Null deviance: 55.30408877787105
Residual deviance: 32.91674782335035
AIC: 225.77296812326398


In [5]:
# Recarregar o dataset para garantir que a coluna está intacta
data_titanic = h2o.import_file("..\\data\\titanic.csv" )

# Converter a variável alvo para fator para o modelo de classificação
data_titanic[target] = data_titanic[target].ascharacter().asfactor()

# Dividir em treino e teste novamente para o modelo de classificação
train, test = data_titanic.split_frame([0.75], seed=42)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
# Treinar o modelo de classificação

model_classification = H2ORandomForestEstimator(seed=42)
model_classification.train(x=predictors, y=target, training_frame=train)
performance_classification = model_classification.model_performance(test)
print("\nResultados do Modelo de Classificação:")
print(performance_classification)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

Resultados do Modelo de Classificação:
ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.138176098127102
RMSE: 0.3717204569661212
LogLoss: 0.4419274883968322
Mean Per-Class Error: 0.18434886499402628
AUC: 0.8733174034249302
AUCPR: 0.8602379439108375
Gini: 0.7466348068498605

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4086468636989593
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      107  28   0.2074   (28.0/135.0)
1      15   78   0.1613   (15.0/93.0)
Total  122  106  0.1886   (43.0/228.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.408647     0.78392   86
max f2                       0.268095     0.82996   102
max f0point5                 0.770314     0.809969  40
max accuracy

## Análise de Utilidade e Performance dos Modelos

Para determinar qual dos modelos é mais útil e apresenta melhor performance preditiva, analisamos suas características em relação ao objetivo do exercício.

### 1. Utilidade dos Modelos
   - **Modelo de Regressão**: Este modelo prevê um valor contínuo, que pode ser interpretado como uma probabilidade de sobrevivência. Ele é útil se estivermos interessados em uma medida da "intensidade" com que um passageiro teria chances de sobreviver. No entanto, para um problema de classificação binária (como prever sobrevivência `sim/não`), a utilidade desse modelo é limitada, pois não fornece uma resposta categórica direta.
   - **Modelo de Classificação**: O modelo de classificação é mais apropriado para problemas binários, como este, onde queremos prever uma categoria (sobreviveu ou não). Este modelo fornece uma probabilidade de pertencer a cada classe, permitindo uma decisão clara, o que torna o modelo de classificação mais útil para prever a sobrevivência dos passageiros.

### 2. Performance Preditiva
   - A performance dos modelos pode ser comparada com métricas como **AUC** (Área Sob a Curva), **Log Loss** e **Accuracy**.
   - O **AUC** é especialmente útil para modelos de classificação, pois mede a capacidade do modelo de distinguir entre classes. Em problemas de classificação binária, essa métrica indica a eficácia do modelo em prever a classe correta.
   - Para determinar o melhor modelo em termos de performance, deve-se observar qual apresenta o maior **AUC** e o menor **Log Loss** no conjunto de teste. A **Accuracy** também é relevante se o foco for a taxa de predições corretas, embora o AUC seja, em geral, mais informativo para avaliar classificadores.

### Conclusão Recomendada
- **Modelo Mais Útil**: O modelo de classificação (Random Forest) é o mais útil para o caso em questão, pois gera predições categóricas diretas de sobrevivência.
- **Melhor Modelo em Performance**: Avalie as métricas dos dois modelos e selecione aquele com o maior **AUC** (e, se necessário, confirme com a **Accuracy** e **Log Loss**). Para problemas de classificação binária, o modelo de classificação tende a ter uma performance preditiva superior em termos de AUC e precisão categórica em comparação com o modelo de regressão.

Em resumo, o modelo de classificação é o mais vantajoso para uma implementação prática, pois alinha-se com o objetivo de prever diretamente se o passageiro sobreviveu ou não.
